# Create sagemaker endpoints
This notebook creates sagemaker endpoints with the pretrained models

In [ ]:
import boto3
import sagemaker
import os
import numpy as np
import urllib.request

endpoint_instance_type = "SAGEMAKER_INFERENCE_INSTANCE_TYPE"
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()
role = sagemaker.get_execution_role()

### Make your own S3 bucket and copy the pre-trained models into your own bucket
Package the pretrained models

An s3 bucket is a cloud storage. In this section, we will make a bucket in your AWS account then copy the pre-trained models into your own bucket

In [ ]:
s3_client = boto3.client('s3')
s3_client.upload_file("rllib_inference/model.tar.gz", s3_bucket, 
                      "battlesnake-aws/pretrainedmodels/model.tar.gz")

# Create a sagemaker endpoint with the bucket


In [ ]:
model_data = "s3://{}/battlesnake-aws/pretrainedmodels/model.tar.gz".format(s3_bucket)
print("Make an endpoint with {}".format(model_data))

Copy heuristics into the folder

In [ ]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data=model_data,
              role=role,
              entry_point="inference.py",
              source_dir='rllib_inference/src',
              framework_version='2.1.0',
              name="battlesnake-rllib",
             )

# Deploy an inference endpoint
predictor = model.deploy(initial_instance_count=1, instance_type=endpoint_instance_type, endpoint_name='battlesnake-endpoint')

# Test the model

In [ ]:
import numpy as np
from time import time

data1 = np.zeros(shape=(1, 21, 21, 12), dtype=np.float32).tolist()

health_dict = {0: 50, 1: 50}
json = {"turn": 4,
        "board": {
                "height": 15,
                "width": 15,
                "food": [],
                "snakes": []
                },
            "you": {
                "id": "snake-id-string",
                "name": "Sneky Snek",
                "health": 90,
                "body": [{"x": 1, "y": 3}]
                }
            }

before = time()
action = predictor.predict({"state": data1, "prev_action": -1, 
                           "prev_reward": -1, "seq_lens": -1,  
                           "all_health": health_dict, "json": json})
elapsed = time() - before

action_to_take = action["outputs"]["heuristisc_action"]
print("Action to take {}".format(action_to_take))
print("Inference took %.2f ms" % (elapsed*1000))